# Manage pathway ontology

## Background

Pathways in single-cell analysis represent the interconnected networks of molecular signaling cascades that govern critical cellular processes. They are of utmost importance as they offer a comprehensive understanding of the intricate regulatory mechanisms underlying cellular behavior, providing insights into disease pathogenesis, therapeutic responses, and the identification of potential targets for precision medicine and intervention strategies.

Managing pathways across different datasets is crucial in a biotech company to gain a comprehensive understanding of complex biological processes and facilitate efficient research and development.

In this notebook we are registering the 2023 GO Biological process pathway ontology with Lamin. Afterwards, we are linking the pathways to genes and conducting a pathway enrichment analysis on an interferon-beta treated dataset. Finally, we will demonstrate how to fetch datasets with pathway queries using Lamin.

## Setup

In [ ]:
import gseapy as gp
import scanpy as sc
import matplotlib.pyplot as plt

from lamin_examples import datasets as ds

```{warning}

Please ensure that you have created or loaded a LaminDB instance before running the remaining part of this notebook!
```

In [ ]:
# A lamindb instance containing bionty schema (skip if you already loaded your instance)

!lamin init --storage enrichr --schema bionty

In [ ]:
import lamindb as ln
from lnschema_bionty import Pathway, Gene

## Fetch `GO_Biological_Process_2023` pathways annotated with human genes using Enrichr

First we fetch the "GO_Biological_Process_2023" pathways for humans using [GSEApy](https://github.com/zqfang/GSEApy) which wraps [GSEA](https://www.gsea-msigdb.org/gsea/index.jsp) and [Enrichr](https://maayanlab.cloud/Enrichr/).

In [ ]:
go_bp = gp.get_library(name="GO_Biological_Process_2023", organism="Human")
print(f"Number of pathways {len(go_bp)}")

In [ ]:
go_bp["ATF6-mediated Unfolded Protein Response (GO:0036500)"]

Parse out the ontology_id from keys, convert into the format of {ontology_id: (name, genes)}

In [ ]:
def parse_ontology_id_from_keys(key):
    """Parse out the ontology id.

    "ATF6-mediated Unfolded Protein Response (GO:0036500)" -> ("GO:0036500", "ATF6-mediated Unfolded Protein Response")
    """
    id = key.split(" ")[-1].replace("(", "").replace(")", "")
    name = key.replace(f" ({id})", "")
    return (id, name)

In [ ]:
go_bp_parsed = {}

for key, genes in go_bp.items():
    id, name = parse_ontology_id_from_keys(key)
    go_bp_parsed[id] = (name, genes)

In [ ]:
go_bp_parsed["GO:0036500"]

## Register pathway ontology in LaminDB

In [ ]:
pathway_bionty = Pathway.bionty()  # equals to bionty.Pathway()
pathway_bionty

Next, we register all the pathways and genes in LaminDB to finally link pathways to genes.

### Register pathway terms

To register the pathways we make use of {func}`docs:lamindb.parse` to directly parse the annotated GO pathway ontology IDs into LaminDB.

In [ ]:
pathway_records = ln.parse(go_bp_parsed.keys(), Pathway.ontology_id)

In [ ]:
pathway_records[:3]

In [ ]:
ln.save(pathway_records);

### Register gene symbols

Similarly, we use {func}`docs:lamindb.parse` for all Pathway associated genes to register them with LaminDB.

In [ ]:
all_genes = {g for genes in go_bp.values() for g in genes}

In [ ]:
gene_records = ln.parse(all_genes, Gene.symbol)

In [ ]:
gene_records[:3]

In [ ]:
ln.save(gene_records);

### Link pathway to genes

Now that we are tracking all pathways and genes records, we can link both of them to make the pathways even more queryable.

In [ ]:
gene_records_ids = {record.symbol: record for record in gene_records}

In [ ]:
for pathway_record in pathway_records:
    pathway_genes = go_bp_parsed.get(pathway_record.ontology_id)[1]
    pathway_genes_records = [gene_records_ids.get(gene) for gene in pathway_genes]
    pathway_record.genes.set(pathway_genes_records)

Now genes are linked to pathways:

In [ ]:
pathway_record.genes.values_list("symbol", flat=True)

## A interferon-beta treated dataset

We will now conduct a pathway enrichment analysis on a small peripheral blood mononuclear cell dataset that is split into control and stimulated groups.
The stimulated group was treated with interferon beta.

The dataset was initially obtained using `From "SeuratData::ifnb"`.

Let's load the dataset and look at the cell type annotations.

In [ ]:
adata = ds.anndata_seurat_ifnb()
adata

In [ ]:
adata.obs["seurat_annotations"].value_counts()

For simplicity, we subset to "B Activated" cells:

In [ ]:
adata_ba = adata[adata.obs.seurat_annotations == "B Activated"].copy()
adata_ba

### Pathway enrichment analysis using Enrichr

This analysis is based on: https://gseapy.readthedocs.io/en/master/singlecell_example.html

First, we compute differentially expressed genes using a Wilcoxon test between stimulated and control cells.

In [ ]:
# compute differentially expressed genes
sc.tl.rank_genes_groups(
    adata_ba,
    groupby="stim",
    use_raw=False,
    method="wilcoxon",
    groups=["STIM"],
    reference="CTRL",
)

rank_genes_groups_df = sc.get.rank_genes_groups_df(adata_ba, "STIM")

In [ ]:
rank_genes_groups_df.head()

Next, we filter out up/down-regulated differentially expressed gene sets:

In [ ]:
degs_up = rank_genes_groups_df[
    (rank_genes_groups_df["logfoldchanges"] > 0)
    & (rank_genes_groups_df["pvals_adj"] < 0.05)
]
degs_dw = rank_genes_groups_df[
    (rank_genes_groups_df["logfoldchanges"] < 0)
    & (rank_genes_groups_df["pvals_adj"] < 0.05)
]

In [ ]:
degs_up.shape, degs_dw.shape

Run pathway enrichment analysis on DEGs and plot top 10 pathways:

In [ ]:
enr_up = gp.enrichr(degs_up.names, gene_sets="GO_Biological_Process_2023").res2d

gp.dotplot(enr_up, figsize=(2, 3), title="Up", cmap=plt.cm.autumn_r);

In [ ]:
enr_dw = gp.enrichr(degs_dw.names, gene_sets="GO_Biological_Process_2023").res2d

gp.dotplot(enr_dw, figsize=(2, 3), title="Down", cmap=plt.cm.winter_r, size=10);

## Track datasets containing annotated pathways in LaminDB

Let's enable tracking of the current notebook as the transform of this file:

In [ ]:
ln.track()

We further create a {func}`docs:lamindb.File` object to track the dataset.

In [ ]:
file = ln.File(adata_ba, name="seurat_ifnb_activated_Bcells")

In [ ]:
ln.save(file)

We further create two {func}`docs:lamindb.Featureset` for `degs_up` and `degs_dw` which we can later associate with the associated pathways:

In [ ]:
degs_up_featureset = ln.FeatureSet.from_iterable(degs_up.names, Gene.symbol)

In [ ]:
degs_dw_featureset = ln.FeatureSet.from_iterable(degs_dw.names, Gene.symbol)

In [ ]:
ln.save(degs_up_featureset)
ln.save(degs_dw_featureset);

Link the top 10 pathways to the corresponding differentially expressed genes:

In [ ]:
# get ontology ids for the top 10 pathways
enr_up_top10 = [
    pw_id[0] for pw_id in enr_up.head(10).Term.apply(parse_ontology_id_from_keys)
]
enr_dw_top10 = [
    pw_id[0] for pw_id in enr_dw.head(10).Term.apply(parse_ontology_id_from_keys)
]

# get pathway records
enr_up_top10_pathways = ln.parse(enr_up_top10, Pathway.ontology_id)
enr_dw_top10_pathways = ln.parse(enr_dw_top10, Pathway.ontology_id)

Associate the pathways to the differentially expressed genes:

In [ ]:
degs_up_featureset.pathways.set(enr_up_top10_pathways)
degs_dw_featureset.pathways.set(enr_dw_top10_pathways)

In [ ]:
degs_up_featureset.pathways.values_list("name", flat=True)

Link the file to features:

In [ ]:
file.featuresets.add(degs_up_featureset)
file.featuresets.add(degs_dw_featureset)

## Querying for pathways

Querying for pathways is now simple with {func}`docs:lamindb.select`:

In [ ]:
ln.select(Pathway).filter(name__contains="interferon-beta").df()

Query pathways from a gene:

In [ ]:
ln.select(Pathway).filter(genes__symbol="KIR2DL1").df()

Query files from a pathway:

In [ ]:
ln.select(ln.File).filter(
    featuresets__pathways__name__icontains="interferon-beta"
).first()

Query featuresets from a pathway to learn from which geneset this pathway was computed:

In [ ]:
pathway = ln.select(Pathway, ontology_id="GO:0035456").one()
pathway

In [ ]:
degs = ln.select(ln.FeatureSet).filter(pathways__ontology_id=pathway.ontology_id).one()

Now we can get the list of genes that are differentially expressed and belong to this pathway:

In [ ]:
pathway_genes = set(pathway.genes.values_list("symbol", flat=True))
degs_genes = set(degs.genes.values_list("symbol", flat=True))

In [ ]:
pathway_genes.intersection(degs_genes)

## Conclusion

Registering pathways and associated gene sets is made simple with {func}`docs:lamindb.parse` that ensures that all parsed objects are linked to ontology IDs.Linking both sets is possible with {func}`docs:lamindb.Featureset` to facilitate simple querying for datasets that contain specific pathways. Since the pathways are linked to genes, Lamin also enables fetching the associated genes of a registered pathway to, for example, retrieve sets of differentially expressed genes that are a part of a specific pathway.

## Try it yourself

This notebook is available at [https://github.com/laminlabs/lamin-examples](https://github.com/laminlabs/lamin-examples).